In [1]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.Collecting mysql-connector-python
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   --- ------------------------------------ 1.6/16.1 MB 10.5 MB/s eta 0:00:02
   --------- ------------------------------ 3.9/16.1 MB 10.7 MB/s eta 0:00:02
   --------------- ------------------------ 6.3/16.1 MB 11.0 MB/s eta 0:00:01
   ---------------------- ----------------- 8.9/16.1 MB 11.1 MB/s eta 0:00:01
   --------------------------- ------------ 11.0/16.1 MB 10.9 MB/s eta 0:00:01
   -------------------------------- ------- 13.1/16.1 MB 10.7 MB/s eta 0:00:01
   ------------------------------------- -- 15.2/16.1 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 16.1/16.1 MB 10.3 MB/s eta 0:00:00




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\bruno\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import mysql.connector

In [2]:
import getpass  # To get the password without showing the input
password = getpass.getpass()

In [11]:
# Se crea una conexión con mysql

conn = mysql.connector.connect(host='localhost',
                        user='root',
                        passwd=password,
                        database='sakila'
                        )

In [12]:
# Se activa el cursor

cursor = conn.cursor()

cursor

In [13]:
cursor.close()
conn.close()

In [16]:
import pymysql
from sqlalchemy import create_engine, text
import pandas as pd
from urllib.parse import quote_plus

def rentals_month(engine, month, year):
    # Crear una consulta SQL para recuperar los datos de alquiler para el mes y año especificados
    query = text("""
    SELECT rental_id, rental_date, inventory_id, customer_id, return_date, staff_id, last_update
    FROM rental
    WHERE EXTRACT(MONTH FROM rental_date) = :month AND EXTRACT(YEAR FROM rental_date) = :year
    """)

    # Ejecutar la consulta y recuperar los datos en un DataFrame de pandas
    with engine.connect() as connection:
        result = connection.execute(query, month=month, year=year)
        rentals_df = pd.DataFrame(result.fetchall(), columns=result.keys())

    return rentals_df

# Parámetros de conexión
host = 'localhost'
user = 'root'
password = password
database = 'sakila'

# Codificar usuario y contraseña
user_quoted = quote_plus(user)
password_quoted = quote_plus(password)

# Crear un engine de SQLAlchemy utilizando la conexión pymysql
engine = create_engine(f'mysql+pymysql://{user_quoted}:{password_quoted}@{host}/{database}')

# Usar el engine en la función rentals_month
month1 = 5  # Por ejemplo, noviembre
month2 = 6 
year = 2005
df1 = rentals_month(engine, month1, year)
df2 = rentals_month(engine, month2, year)
print(df2)


      rental_id         rental_date  inventory_id  customer_id  \
0          1158 2005-06-14 22:53:33          1632          416   
1          1159 2005-06-14 22:55:13          4395          516   
2          1160 2005-06-14 23:00:34          2795          239   
3          1161 2005-06-14 23:07:08          1690          285   
4          1162 2005-06-14 23:09:38           987          310   
...         ...                 ...           ...          ...   
2306       3465 2005-06-21 22:10:01          1488          510   
2307       3466 2005-06-21 22:13:33           371          226   
2308       3467 2005-06-21 22:19:25           729          543   
2309       3468 2005-06-21 22:43:45          2899          100   
2310       3469 2005-06-21 22:48:59          4087          181   

             return_date  staff_id         last_update  
0    2005-06-18 21:37:33         2 2006-02-15 21:30:53  
1    2005-06-17 02:11:13         1 2006-02-15 21:30:53  
2    2005-06-18 01:58:34         2 2

In [20]:
import pandas as pd

def rental_count_month(df, month, year):
    # Crear una columna de conteo de alquileres
    rental_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{str(month).zfill(2)}_{year}')
    
    return rental_count

# Ejemplo de uso
# Asumiendo que ya tienes un DataFrame df proporcionado por rentals_month
month1 = 5
month2 = 6
year = 2005
#df = rentals_month(engine, month, year)  # Supongamos que esta es la llamada a tu función anterior

result_df1 = rental_count_month(df1, month1, year)
result_df2 = rental_count_month(df2, month2, year)
print(result_df1)


     customer_id  rentals_05_2005
0              1                2
1              2                1
2              3                2
3              5                3
4              6                3
..           ...              ...
515          594                4
516          595                1
517          596                6
518          597                2
519          599                1

[520 rows x 2 columns]


In [23]:
import pandas as pd

def compare_rentals(df1, df2, column1, column2):
    # Combinar los dos DataFrames en uno solo utilizando la columna 'customer_id' como clave
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_month1', '_month2'))
    
    # Rellenar los valores NaN con 0 para poder realizar la diferencia correctamente
    combined_df = combined_df.fillna(0)
    
    # Crear la nueva columna 'diferencia' que representa la diferencia entre los dos meses
    combined_df['diference'] = combined_df[column1] - combined_df[column2]
    
    return combined_df

# Ejemplo de uso
# Asumiendo que tienes dos DataFrames: df_month1 y df_month2
# df_month1 tiene una columna llamada 'rentals_05_2024'
# df_month2 tiene una columna llamada 'rentals_06_2024'
#df_month1 = pd.DataFrame({'customer_id': [1, 2, 3], 'rentals_05_2024': [5, 7, 10]})
#df_month2 = pd.DataFrame({'customer_id': [1, 2, 3], 'rentals_06_2024': [6, 5, 12]})

result_df = compare_rentals(result_df1, result_df2, 'rentals_05_2005', 'rentals_06_2005')
print(result_df)


     customer_id  rentals_05_2005  rentals_06_2005  diference
0              1              2.0              7.0       -5.0
1              2              1.0              1.0        0.0
2              3              2.0              4.0       -2.0
3              5              3.0              5.0       -2.0
4              6              3.0              4.0       -1.0
..           ...              ...              ...        ...
593          583              0.0              6.0       -6.0
594          585              0.0              4.0       -4.0
595          591              0.0              3.0       -3.0
596          592              0.0              5.0       -5.0
597          598              0.0              1.0       -1.0

[598 rows x 4 columns]


In [19]:
!pip install sqlalchemy
!pip install pymysql


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\bruno\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\bruno\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
